In [ ]:
import pandas as pd
import numpy as np
import math
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import cross_val_score

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

# Grid Search, Hyperparameters, and Cross Validation

A large part of setting up machine learning models involves repetitive trials, and trial and error tests. We have several tools that can largely automate this trial process, saving us the trouble of making repetitve loops and managing large lists of test values and results. 

One of the things that we can use this for is to find good values for what we call Hyperparamaters - or the values that steer the model creation process. 


### What the Heck is a Hyperparameter?

A hyperparameter is a futuristic term for a simple concept - hyperparameters are the settings that dictate how a machine learning algorithm will behave during training. Hyperparameters are generally the arguments that we provide when creating a model in code. 

Hyperparameters can be contrasted with regular parameters:
<ul>
<li> Hyperparameters are defined <b>outside</b> of the training process and control how that process works. To adjust a hyperparameter we need to do it by "hand" (more on this later), the algorithm can't figure out the optimal setting during training. 
<li> Parameters exist <b>inside</b> the training process, and these are adjusted by the algorithm during training. The parameters are things like the slopes and intercept that are generated by a linear/logsitic regression. 
</ul>

This means that to get an accurate model we generally have two processes of determining the "best" model, one that we handle manually (hyperparameters), and one that is done within the training process. 

In [ ]:
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

df = sklearn_to_df(sklearn.datasets.load_breast_cancer())
df.head()

<h4>Generate Decision Tree Model</h4>

Create a model with the above data and display the resulting decision tree. 

The random state thing is to make it generate the same results each run, that's not really needed in general, but trees can be very different on each training run, so this will make things a little more stable as we explore. 

In [ ]:
df2 = pd.get_dummies(df, drop_first=True)
y = np.array(df2["target"]).reshape(-1,1)
X = np.array(df2.drop(columns={"target"}))

X_train, X_test, y_train, y_test = train_test_split(X, y)

clf = DecisionTreeClassifier(random_state=10)
clf = clf.fit(X_train, y_train)
print(clf.get_depth())
print(clf.score(X_test, y_test))
plot_tree(clf)

### Changing the Model's Function

#### What if we want to limit max depth, or similar?

Now, if we wanted to run a trial with different max depths we could just change the number in the argument and repeat. If we wanted to get a little more sophisticated, we could write a loop to try a few and see the results. However, what if we want to change several of these hyperparamaters in different trials? Writting a nested loop for each different value we're changing could become pretty cumbersome, and the more clunky it is, the more likely we are to make a mistake. 

### Hyperparamaters and GridSearchCV

Luckily, this is a common task, so there are prebuilt ways to do it efficiently. What we are doing here is Hyperparamater Tuning, and the tool we'll use is called GridSearchCV, from sklearn. GridSearchCV is effectively a shortcut to nested loops for each hyperparamater - we provide a list of which variables we'd like to change, and which values we'd like the system to try, and it does the messy work of running a bunch of trials behind the scenes for us. Much easier to use, and less prone to making a mistake. 

For this trial, we'll change the max depth and also the min_samples_split value - this hyperparamater sets how many values must be in one node before it can be split into 2. The default is 2, and this may sometimes lead to overfitting - we can try a few other options. 

#### Using Grid Search

A grid search object fits into the other pipeline objects that we've used before, in a similar way. We specify:
<ul>
<li> Which algorithm to train/fit. 
<li> Which hyperparameters to try, and which values to test. 
<li> The number of cross validation trials (more on this in a moment)
</ul>

The grid search object will automatically run the entire process that we are used to in training a model - train/test split of the data, train model, check accuracy - but it will do it for each combination of hyperparameter options that we defined in the lists.

For the example below, we specified the maximum depth and the minimum number of items before a node can be split, these are two hyperparameters that control how the model's training process runs. The grid search will do the entire process for each possible combination of those values we listed, check the accuracy of the resulting model, and eventually give us the best one. 

The grid search process is really useful, we don't need to manually guess and test a bunch of different combination of hyperparameters, we can set the potential options and just let it run. In theory, if we were to set every possible option for every possible hyperparameter we'd have an automated way of finding the absoulte most accurate set of HP to use, however that would be pretty impractical in terms of time to run. We want to narrow down the possibilities a bit using both our judegement and the results of the model in ititial trials and exploration. For example, in this one we can see that the trail run has many layers and many nodes with very small numbers that are split, so that's a good indication we may want to limit those in our grid search. The exact options and what we 'should' try will differ depending on the algorithm we are using. SK Learn also has some varaiations on the grid search that aim to make the process faster, they implement things like random trials of hyperparameter values or slicing down the potential HP values to try. We won't spend much time looking at these, the concept is the same as the original grid search just with a bit of variance in how the list of HP values to try; you can test one out pretty easily once comfortable with the grid search. 

In [ ]:
from sklearn.model_selection import GridSearchCV

#Create a dictionary of HP names and lists of potential values. 
#The grid search will try all of these options. 
tree_para = {'min_samples_split':[2,3,4,5,6,7],'max_depth':[3,4,5,6,7,8]}

clf = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=tree_para, cv=10) #See below for the CV argument
clf.fit(X_train, y_train)
clf.best_estimator_

Voila!!

There is not much of a downside to trying every possible combination of every HP, especially for us with our small datasets. The main drawback is that this can become REALLY computationally expensive with lots of options and large amounts of data. The growth in calculation time grows exponentially the more options are added, so for real world datasets just "trying everything" is probably prohibitably slow. 

There are some ways to cut down on useless trials, some are listed in the sklearn documentation. We'll look at making this better more in the future in a little more depth. For large datasets, taking a sample to experiment on is probably a prudent move. 

## Cross Validation

The CV argument above stands for cross validation - another way to prevent overfitting and increase overall accuracy. Cross validation is basically a supercharged test-train split. 

The standard test-train split divides the data 70/30ish, and calculating the accuracy using that reserved 30ish percent. 

Cross validation makes K splits in the data, and repeats this test-train calculation K times. Each time a different subset is the test set, and the rest of the data is the training set. The overall score is the average of all those K trials. There are other methods in addition to K-fold, such as Leave-One-Out, which leaves only 1 example in the test set, and others that stratify to deal with target imbalance. Using these is pretty similar, we'll touch on them later. 

Cross validation is a very common step in machine learning. It reduces variance in the results as any random difference in the test-train split is negated by all the other samples. In general, cross validation will give more reliable results due to that reduction in variance - that effect being greater when there are outliers (think if one outlier is tested in a linear regression, that amount of error may be substantial). In testing the accuracy of models previously we'd occasionally see wild variations in the results due to randomness in how the data ended up being split, this cross validation process (largely) neutralizes that issue. 

We will use a separate cross validation in the future, it is also in sklearn and is pretty straightforward. We commonly use it as part of the grid search as we have here, since both are common this makes it easier and more compact. There are also a few more options that we'll tweak later on, but they are pretty simple. 

<h3>Why is There Still a Test Set?</h3>

The image (from sklearn), and the code below, employ both the test-train split and the cross validation. This is the technically correct way to do it. We want to test on data that the model has never encountered in its training. In this setup, the test set is the same as it always way, and the cross validation creates a bunch of validation splits that help train the data. In practice, if you didn't do this it would rarely have a big impact on the outcome. Examples across the internet will show it both ways, don't worry about it too much, it is a more theoretical concern for our purposes.

This does bring up one point that matters when looking to actually use one of these models. All of the work we've done so far has been on building and evaluating a predicitve model. When that's finished, we'd probably want to use it in production. When this happens we normally want to use all of our data (no test-train split) to create the final model. The idea is that we've already determined that the given algorithm and hyperparamaters are good in creating a model, now we want to create a version without that test data withheld so we can benefit from all the data in making the final model. We won't have a real test score for this model, as there is no leftover data to test it on, but we can pretty safely assume that it'll be at least as good as the one we built with a split. 

![Cross Validation](images/grid_search_cross_validation.png "Cross Validation" )

<b>From the sklearn documentation:</b>

> When evaluating the resulting model it is important to do it on held-out samples that were not seen during the grid search process: it is recommended to split the data into a development set (to be fed to the GridSearchCV instance) and an evaluation set to compute performance metrics.

### Cross Validation Scoring

We can use cross validation scoring on its own in place of normal scoring very easily to get a cross-validated version of the training scores, just replace the score function with cross_val_score. The result becomes an array of all the scores. 

A K value of 5 to 10 is pretty typical. 

In [ ]:
#Cross validation scoring
crossV = DecisionTreeClassifier()
scores = cross_val_score(crossV, X, y, cv=5)
print(scores)
print(np.mean(scores))

#### Exercise - Make Your Own Grid

Try to make a similar gridsearch with some differnent hyperparamaters. Look on the documentation page in sklearn and try a few options. 

In [ ]:
#EXERCISE

### Grid Search and Cross Validation

One note to pay attention to as we start using these tools is that they can easily cause the amount of processing time needed to explode, since each trial is effectively a full end-to-end training process. Taking care to limit the number of options in a grid search, capping the number of cross validation trials, and performing initial work with a sample of the full data can all help prevent the time from running away. There are no strict rules on "how much stuff to try", if things take forever to run, start dialing some stuff back. 

## Ensamble Models - Random Forest

One common improvement made when using trees is to use several of them, or make a forest instead of a tree. More generally, this is our first look at an ensamble model - combining two or more predictive models to make a final prediction. 

The most simple way to create an ensamble is to use a Random Forest, which generates several trees that make predictions in parallel, then those predictions are combined (for a classification, majority wins) to make the final prediction. The magic behind this is that we are creating several uncorrelated predictions - each tree is generated from a random subset of the data. 

Random forests employ an ensamble method called Bagging, meaning that we run a bunch of models parallel to each other and get the result by combining the individual results; i.e. all the models are in a bag, and we pull out their combined answer. Another ensamble method is called boosting - using the results of one model's predictions to improve another's; we will look at boosting later on. 

Random forests will generally deliver more accuracy and less overfitting than a singular tree. More specifically it will tend to reduce the varaince in the predictions, as overfitting tends to 'cancel out' (sklearn's words) when many trees are created independently.

![Random Forest](images/random_forest.png "Random Forest" )

Using a Ramdom Forest is pretty easy, we can try one with a grid search, and up the number of HPs that we are tuning. Check the documentation page for a look at the paramaters that we could use: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html 

<b>Note:</b> the n_jobs argument is optional, that tells the system to create that number of threads, and those threads will be run in parallel on different cores of the computer. Modern CPUs have a bunch of cores 4-16 or more, so there's potential for some speedup there. My work Macbook I made this on has 4 (which I think can each double thread, to 8). If you get a weird error, just remove this. This isn't really a need to know, but it may speed some stuff up here and there. If someone were doing this on a powerful server you might have 64 cores, so you'd set that number to be high to ensure that calculations went in parallel on each one. 

### Bootstrapping

One key step that we have when using a random forest is bootstrapping, or making the datasets that each of those models in the ensable use. Bootstrapping generates random sets of data for each model by selecting data from the full training set. Each dataset is the same size as the original data, but it contains items that were selected randomly, with replacement. I.e. if our dataset is 100 items, to create each dataset while bootstrapping we would randomly pick an item from the full data, put that value in the boostrap set, leave the original in the full dataset (the original data stays at 100 items), and repeat that 100 times. The end result are a bunch of datasets that are very similar to the original, but slightly, and randomly, different. 

This bootstrapping step generates datasets that help the models be more resistant to overfitting, in simple terms it is harder for a model to overfit if the data it is being fit to is different in each of many (in the case of a forest, often 100) models. While one tree model might be able to overfit to its training data, there are very low odds of all 100 models getting a dataset that allows the same overfitting to happen, so the overfit models will be "overruled" when the final vote happens. 

### Feature Sampling

Ensables can also apply the same bootstrapping idea to the feature set - rather than use all the features to train the models, take some randomized subset for each model. This is kind of an odd concept, as we are purposefully removing data, but it can be helpful, mainly in combatting overfitting. We'll examine this idea a bit more when looking at neural networks and their ability to use a "dropout" to remove individual features from the training data at random.  

### Be Weak to be Strong

Ensable models bring up a concept that we will focus on in more depth when we look at boosting ensables and neural networks later on, the idea of weak and strong learners. The differentiation between weak and strong predictive models are (this is in the context of classifcation models):
<ul>
<li> <b>Strong Learners:</b>
    <ul>
    <li> Perform much better than random guessing. 
    <li> Take "more effort" to train and run. 
    <li> Most of the models we look at will fall into this category - logistic regression, SVM, etc...
    </ul>
<li> <b>Weak Learners:</b>
    <ul>
    <li> Perform slightly better than random guessing. 
    <li> Take little time/effort to train or use. 
    <li> "Small" versions of models such as 1 layer trees. 
    </ul>
</ul>

This distinction seems kind of odd at first glance, as there doesn't really seem to be much of a reason to ever use or care about a weak learner - why do we want something that does worse? Ensable models seek to overcome this by assembing multiple weak (or weaker) models together, taking advantage of the speed and non-overfitting of weak models to generate results that are (ideally) better than a standard strong model. The ensables we will look at now like a random forest normally combine a bunch of slightly "weaker" (usually not 1 layer, but normally not a massively large tree) models together to create a better model. When we look at boosted (another ensable technique) ensables, we'll spend more time on this. 

In [ ]:
rf_para = {'min_samples_split':[3,4,5,6,7],
            'max_depth':[3,4,5,6,7,8],
            'n_estimators':[20,50,80],
            'criterion':["gini","entropy"],
            'max_samples':[.4, .5, .6, .7]}

#clf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=rf_para, cv=10)
# If you get some weird error, uncomment above, and comment the line below. See the note above for more. 
 
clf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=rf_para, cv=10, n_jobs=4) 
clf.fit(X_train, y_train.ravel())
clf.best_estimator_

### Pipeline with GridSearchCV

We can also use this with a pipeline, the main changes are:
<ul>
<li>The estimator in the grid search becomes the pipeline.
<li>The paramaters in the param_grid need to be named "stepName__variable". See the example below.
</ul>

This can be extened pretty much indefinitely, we can have a pipelines as part of a column transformer, as part of another pipeline, etc... all plugged into the Grid Search. As long as calling .fit() on whatever we put into the grid search is valid. 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
estimator = RandomForestClassifier(n_jobs=-1)
pipe = Pipeline(steps=[("scaler", scaler), ("forrest", estimator)])

params = {'forrest__max_depth':[5,6,7,8,9]}
 
clf = GridSearchCV(pipe, param_grid=params, cv=5, n_jobs=-1) 
clf.fit(X_train, y_train.ravel())
best = clf.best_estimator_
print(best.score(X_test, y_test))
print(best)

## Non-Tree Bagging

In general the BaggingClassifier in sklearn can apply the same concept to other classification methods, and the BaggingRegressor can do the same for regression. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier

bag = BaggingClassifier(base_estimator=LogisticRegression(max_iter=10000),n_estimators=10, random_state=0).fit(X_train, y_train.ravel())
bag.score(X_test, y_test)

## Exercise - Predict the Target


In [ ]:
df_ = sklearn_to_df(sklearn.datasets.load_iris())
df_.head()

In [ ]:
#Model
